In [ ]:
from spyral.core.run_stacks import form_run_string
from spyral_utils.plot import Histogrammer
from spyral_utils.nuclear import NuclearDataMap
from spyral_utils.nuclear.particle_id import deserialize_particle_id

import numpy as np
import polars as pl
import scipy
import matplotlib.pyplot as plt
from pathlib import Path

%matplotlib widget

In [ ]:
# Load config
workspace_path = Path("D:\\e20009_analysis")
estimation_result_path = workspace_path / "Estimation"

# Set the run range (inclusive)
run_min = 108
run_max = 366

# Set run to normalize gain match factors to
norm_to_run = 347

# Directory to write gain match factors csv to
output = Path("C:\\Users\\zachs\\Desktop\\e20009_analysis\\e20009_analysis\\e20009_parameters")

In [ ]:
# Parameters for finding gain match factors
pid_x_axis = "sqrt_dEdx" # This is the PID x-axis, matching a column name in the estimation dataframe
pid_y_axis = "brho" # This is the PID y-axis, matching a column name in the estimation dataframe

bins = 200      # Number of bins for dEdx histogram
x_low = -10.0   # Smallest bin edge in dEdx histogram
x_high = 100.0      # Largest bin edge in dEdx histogram
y_low = 0.1      # Cluster must have a brho greater than this to be in the histogram
y_high = 0.4     # Cluster must have a brho smaller than this to be in the histogram

smoothing_factor = 10   # Window size for smoothing histogram

In [ ]:
# Make dictionary to store results
results: dict[str, list] = {"run": [], "gain_factor": []}

In [ ]:
# Find gain match factor of each run
for run in range(run_min, run_max + 1):
    df = None
    try:
        df: pl.DataFrame = pl.read_parquet(
            estimation_result_path / f"{form_run_string(run)}.parquet"
        )
    except Exception:
        continue

    # Project brho range of PID onto the dEdx axis
    df = df.filter((pl.col(pid_y_axis) > y_low) 
                   & (pl.col(pid_y_axis) < y_high))
    energy_loss = df.select(pl.col(pid_x_axis)).to_numpy()

    # Histogram the projection of energies
    hist, bin_edges = np.histogram(
        energy_loss, bins=bins, range=(x_low, x_high)
    )
    bin_width = (x_high - x_low) / bins

    # Perform a moving average smoothing via the convolution theorem of the histogram
    window = np.arange(0, bins, 1)
    window_centered = window - (window[-1] + window[0]) / 2
    fil = np.fft.ifftshift(
        np.sinc(window_centered / smoothing_factor)
    )  # Size of points taken for average is denominator
    transformed = np.fft.fft2(hist, axes=(0,))
    hist_smoothed = np.real(np.fft.ifft2(transformed * fil, axes=(0,)))

    # Find largest peak in smoothed histogram
    pks, props = scipy.signal.find_peaks(
        hist_smoothed, distance=1, prominence=1, width=1, rel_height=0.95)
    
    max_peak_centroid = pks[-1] * bin_width + x_low

    results["run"].append(run)
    results["gain_factor"].append(max_peak_centroid)

In [ ]:
# Normalize gain factors to specified run
run_index = results["run"].index(norm_to_run)
results["gain_factor"] = np.round(results["gain_factor"][run_index] / results["gain_factor"], 2)


In [ ]:
# Write the results to a DataFrame
results_df = pl.DataFrame(results)
results_df.write_csv(Path(output) / "gain_match_factors.csv")

In [ ]:
# Load PID for plotting
pid_path = Path("D:\\e20009_analysis\\particle_id.json")

# Nucleus map
nuclear_map = NuclearDataMap()

# Load PID vertices
pid = deserialize_particle_id(pid_path, nuclear_map)
vertices = np.asarray(pid.cut.get_vertices())

In [ ]:
# Plot PID of one run and compare to gain-matched gate
# Set the run
run= 366

# Create histogram
grammer = Histogrammer()
grammer.add_hist2d("particle_id", (400, 400), ((-10.0, 100), (-0.1, 2.5))) # Plot of dEdx vs. Brho (particle ID)

# Fill histograms
try:
    df: pl.DataFrame = pl.read_parquet(
        estimation_result_path / f"{form_run_string(run)}.parquet"
        )
except Exception:
    raise (f"Estimation phase results not found for run {run}!")

grammer.fill_hist2d('particle_id', df.select(pid_x_axis).to_numpy(), df.select(pid_y_axis).to_numpy())

# Plot PID for run alongside the run's gate with the found gain match factor applied
run_index = results["run"].index(run)
gain_factor = results["gain_factor"][run_index]

pid_hist = grammer.get_hist2d("particle_id")
fig, ax = plt.subplots(1,1)
mesh = ax.pcolormesh(pid_hist.x_bins, pid_hist.y_bins, pid_hist.counts, norm='log')
fig.colorbar(mesh, ax=ax)
ax.plot(vertices[:, 0] / gain_factor, vertices[:, 1], color = 'red')
ax.set_title("Particle ID")
ax.set_xlabel(f"{pid_x_axis} Column")
ax.set_ylabel(f"{pid_y_axis} Column")
fig.set_figheight(8.0)
fig.set_figwidth(11.0)
fig.tight_layout()